In [23]:
import tensorflow as tf
import numpy as np
import math
import os
import random
import numpy as np
import pandas as pd
from pandas import DataFrame
import cPickle as pickle
import math
import copy

In [24]:
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

In [4]:
df_train = pd.read_csv('../data/rl_train_set_unscaled.csv')
df_val = pd.read_csv('../data/rl_val_set_unscaled.csv')
df_test = pd.read_csv('../data/rl_test_set_unscaled.csv')

In [5]:
# let the mortality labels  be -1 and 1: -1 for survival
df_train['died_in_hosp'][df_train['died_in_hosp'] == 0] = -1
df_val['died_in_hosp'][df_val['died_in_hosp'] == 0] = -1
df_test['died_in_hosp'][df_test['died_in_hosp'] == 0] = -1

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [6]:
target_feat = list(np.loadtxt('../data/state_features_pred.txt', dtype=str))
target_feat.append('died_in_hosp')

cur_feat = list(np.loadtxt('../data/state_features.txt', dtype=str))
cur_feat.append('iv_input')
cur_feat.append('vaso_input')

In [7]:
# define an action mapping - how to get an id representing the action from the (iv,vaso) tuple
action_map = {}
count = 0
for iv in range(5):
    for vaso in range(5):
        action_map[(iv,vaso)] = count
        count += 1

In [8]:
inv_action_map = {}
for iv in range(5):
    for vaso in range(5):
        inv_action_map[5*iv+vaso] = [iv,vaso]    

In [9]:
# Change the actions: {0: -0.8, 1: -0.4, 2: 0, 3: 0.4, 4: 0.8}. This will help for the policy search, which
# uses a continuous action space
lookup ={0:-0.8, 1:-0.4, 2:0, 3:0.4, 4:0.8}
for dfm in (df_train, df_val, df_test):
    for iv_input in range(5):
        val = lookup[iv_input]
        dfm['iv_input'][dfm['iv_input'] == iv_input] = val
    for vaso_input in range(5):
        val = lookup[vaso_input]
        dfm['vaso_input'][dfm['vaso_input'] == vaso_input] = val

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.


In [118]:
a = np.arange(5)
b =  np.arange(4)
np.hstack([a,b])

array([0, 1, 2, 3, 4, 0, 1, 2, 3])

In [177]:
#  X: (s_t-2, a_t-2,s_t-1, a_t-1, s_t, a_t )
#  Y: (difference between next state and current state (zeros if end of trajectory), mortality)
def make_data_history(df_in):
    df_in = df_in.reset_index()
    X = []
    Y = []
    count_in_traj = 0
    for count,i in enumerate(df_in.index):
        
        # skip the last one; no next state
        if i == df_in.index[-1]:
            break
       
        # if not terminal step in trajectory    
        if df_in.loc[i, 'icustayid'] == df_in.loc[i+1, 'icustayid']:
            count_in_traj += 1
            if count_in_traj >=3:

                target = df_in.loc[i + 1, target_feat] - df_in.loc[i, target_feat]
                target[-1] = df_in.loc[i, 'died_in_hosp']
                Y.append(target)
                state = np.hstack([df_in.loc[i-2, cur_feat], df_in.loc[i-1, cur_feat], df_in.loc[i, cur_feat]])
                X.append(state)
            else:
                continue
        else:
            count_in_traj = 0

        if count % 10000 == 0 and count > 0:
            print count

    return np.array(X),np.array(Y)

In [10]:
#  X: (states, actions)
#  Y: (difference between next state and current state (zeros if end of trajectory), mortality)
def make_data(df_in):
    X = []
    Y = []
    for count,i in enumerate(df_in.index):
        cur_state = df_in.loc[i,cur_feat]
        if i != df_in.index[-1]:
            # if not terminal step in trajectory             
            if df_in.loc[i, 'icustayid'] == df_in.loc[i+1, 'icustayid']:
                target = df_in.loc[i + 1, target_feat] - df_in.loc[i, target_feat]
                target[-1] = df_in.loc[i, 'died_in_hosp']
                Y.append(target)
                X.append(cur_state)
#   #NOTE: Commented out the lines below because this adds weird training examples to the dataset. We don't
#   # want to predicting zero change at the end of trajectories... Include all pts up to T-1
#             else:
#                 # trajectory is finished
#                 target = np.zeros(len(target_feat))
#                 target[-1] = df_in.loc[i, 'died_in_hosp']
#         else:
#             # last entry in df is the final state of that trajectory
#             target = np.zeros(len(target_feat))
#             target[-1] = df_in.loc[i, 'died_in_hosp']

        if count % 10000 == 0 and count > 0:
            print count

    return np.array(X),np.array(Y)

In [184]:
dire = 'converted_data/'
if not os.path.exists(dire):
    os.makedirs(dire)
    
if not os.path.exists(dire + 'X_train_hist.txt'):
    x_train_hist, y_train_hist = make_data_history(df_train)
    np.savetxt(dire + 'X_train_hist.txt',x_train_hist,fmt='%5.4f')
    np.savetxt(dire + 'Y_train_hist.txt',y_train_hist,fmt='%5.4f')
    print "Saved train"

if not os.path.exists(dire + 'X_val_hist.txt'):
    x_val_hist,y_val_hist = make_data_history(df_val)
    np.savetxt(dire + 'X_val_hist.txt',x_val_hist,fmt='%5.4f')
    np.savetxt(dire + 'Y_val_hist.txt',y_val_hist,fmt='%5.4f')
    print "Saved val"
    
if not os.path.exists(dire + 'X_test_hist.txt'):
    x_test_hist, y_test_hist = make_data_history(df_test)
    np.savetxt(dire + 'X_test_hist.txt',x_test_hist,fmt='%5.4f')
    np.savetxt(dire + 'Y_test_hist.txt',y_test_hist,fmt='%5.4f')
    print "Saved test"

30000
40000
50000
60000
70000
80000
90000
100000
110000
130000
140000
150000
160000
Saved train
10000
20000
Saved val
10000
20000
30000
40000
Saved test


In [189]:
dire = 'converted_data/'
if not os.path.exists(dire):
    os.makedirs(dire)
    
if not os.path.exists(dire + 'X_train.txt'):
    x_train_nohist, y_train_nohist = make_data(df_train)
    np.savetxt(dire + 'X_train.txt',x_train_nohist,fmt='%5.4f')
    np.savetxt(dire + 'Y_train.txt',y_train_nohist,fmt='%5.4f')
    print "Saved train"

if not os.path.exists(dire + 'X_val.txt'):
    x_val_nohist,y_val_nohist = make_data(df_val)
    np.savetxt(dire + 'X_val.txt',x_val_nohist,fmt='%5.4f')
    np.savetxt(dire + 'Y_val.txt',y_val_nohist,fmt='%5.4f')
    print "Saved val"
    
if not os.path.exists(dire + 'X_test.txt'):
    x_test_nohist, y_test_nohist = make_data(df_test)
    np.savetxt(dire + 'X_test.txt',x_test_nohist,fmt='%5.4f')
    np.savetxt(dire +    'Y_test.txt',y_test_nohist,fmt='%5.4f')
    print "Saved test"

10000
20000
Saved val


In [ ]:
# Add Gaussian noise to the state features. Commented out for now -- might need to put back in.
# gaussian_shape = df.loc[:, state_features].values.shape
# noise = np.random.normal(0, 0.03, gaussian_shape)
# df.loc[:, state_features] += noise

In [190]:
x_train = x_train_hist
y_train = y_train_hist

x_val = x_val_hist
y_val = y_val_hist

x_test = x_test_hist
y_test = y_test_hist

In [198]:
hidden_1_size = 32
hidden_2_size = 32
class EnvModel():
    def __init__(self):
        self.phase = tf.placeholder(tf.bool)
        
        self.input_size = x_train.shape[1]
        
        self.target_size = 1 # for now, just predict sofa

        self.cur_state = tf.placeholder(tf.float32, shape=[None, self.input_size],name="cur_state")
        self.targets = tf.placeholder(tf.float32, shape=[None, self.target_size],name="target")
        
        self.fc_1 = tf.contrib.layers.fully_connected(self.cur_state, hidden_1_size, activation_fn=tf.nn.relu)
        self.fc_1_bn = tf.contrib.layers.batch_norm(self.fc_1, center=True, scale=True, is_training=self.phase)
        self.fc_2 = tf.contrib.layers.fully_connected(self.fc_1_bn, hidden_2_size, activation_fn=tf.nn.relu)
        self.fc_2_bn = tf.contrib.layers.batch_norm(self.fc_2, center=True, scale=True, is_training=self.phase)
        
        self.output = tf.contrib.layers.fully_connected(self.fc_2_bn, self.target_size, activation_fn = None)
        
        self.loss = tf.reduce_mean(tf.square(self.targets-self.output)) 
        
        self.est_next_state = self.output + self.cur_state
        
        self.reg_lambda = 0.000
        
        self.vars = tf.trainable_variables() 
        self.lossL2 = tf.add_n([ tf.nn.l2_loss(v) for v in self.vars
                    if 'bias' not in v.name ]) * self.reg_lambda    
        
        self.trainer = tf.train.AdamOptimizer(learning_rate=0.001)
        self.update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
        with tf.control_dependencies(self.update_ops):
        # Ensures that we execute the update_ops before performing the model update, so batchnorm works
            self.update_model = self.trainer.minimize(self.loss + self.lossL2)

In [199]:
config = tf.ConfigProto()
config.gpu_options.allow_growth = True  # Don't use all GPUs 
config.allow_soft_placement = True  # Enable manual control

In [200]:
eval_batch = 64
def run_eval(sess, env_mdl):
    start = 0
    end = 0
    net_loss = 0.0
    while start < len(x_val):
        end = min(len(x_val), start+eval_batch)
        x_batch = x_val[start:end]
        y_batch = y_val[start:end]

        # extract out only sofa for target
        y_batch = y_batch[:, 29]
        y_batch = y_batch[:, None]

        # train using the batch
        loss = sess.run(env_model.loss,
                        feed_dict={env_model.cur_state:x_batch,
                                   env_model.targets:y_batch,
                                   env_model.phase:False})
        # update net loss
        net_loss += loss

        # increment start index to inds
        start += eval_batch
    
    avg_eval_loss = net_loss/(len(x_val)/float(eval_batch))
    
    return avg_eval_loss
        

In [201]:
batch_size = 64
num_epochs = 20
load_model = False #Whether to load a saved model.
save_dir = "./env_model_reg/"
save_path = "./env_model_reg/ckpt"#The path to save our model to.
tf.reset_default_graph()
env_model = EnvModel()

saver = tf.train.Saver(tf.global_variables())

init = tf.global_variables_initializer()

trainables = tf.trainable_variables()

#Make a path for our model to be saved in.
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

with tf.Session(config=config) as sess:
    if load_model == True:
        print('Trying to load model...')
        try:
            restorer = tf.train.import_meta_graph(save_path + '.meta')
            restorer.restore(sess, tf.train.latest_checkpoint(save_dir))
            print "Model restored"
        except IOError:
            print "No previous model found, running default init"
            sess.run(init)
    else:
        print("Running default init")
        sess.run(init)
    print("Init done")
    
    for j in range(1,num_epochs+1):
        net_loss = 0.0
        inds = np.random.permutation(x_train.shape[0])
        start_idx = 0
        end_idx = 0
        while start_idx < len(x_train):
            end_idx = min(len(x_train), start_idx+batch_size)
            batch_inds = inds[start_idx:end_idx]
            x_batch = x_train[batch_inds]
            y_batch = y_train[batch_inds]
            
            # extract out only sofa for target
            y_batch = y_batch[:, 29]
            y_batch = y_batch[:, None]
            
            # train using the batch
            _,loss = sess.run([env_model.update_model,env_model.loss], \
            feed_dict={env_model.cur_state:x_batch,
                       env_model.targets:y_batch,
                       env_model.phase:True})
            
            # update net loss
            net_loss += loss
            
            # increment start index to inds
            start_idx += batch_size
        
        saver.save(sess,save_path)
        print("Saved Model, epoch " + str(j))
        
        av_loss = net_loss/(len(x_train)/float(batch_size))
        print("Average training loss per batch is ", av_loss)
        net_loss = 0.0
        
        eval_loss = run_eval(sess, env_model)
        
        print ("Avg val loss at epoch end is " + str(eval_loss))

    

Running default init
Init done
Saved Model, epoch 1
('Average training loss per batch is ', 0.26523895037921069)
Avg val loss at epoch end is 0.215350268387
Saved Model, epoch 2
('Average training loss per batch is ', 0.20340633354757026)
Avg val loss at epoch end is 0.198151301308
Saved Model, epoch 3
('Average training loss per batch is ', 0.19611293475820771)
Avg val loss at epoch end is 0.196184077906
Saved Model, epoch 4
('Average training loss per batch is ', 0.19248868642049743)
Avg val loss at epoch end is 0.193708232205
Saved Model, epoch 5
('Average training loss per batch is ', 0.1899638535510817)
Avg val loss at epoch end is 0.189579004633
Saved Model, epoch 6
('Average training loss per batch is ', 0.18778610130310694)
Avg val loss at epoch end is 0.189958125447
Saved Model, epoch 7
('Average training loss per batch is ', 0.1861304297970536)
Avg val loss at epoch end is 0.187830319143
Saved Model, epoch 8
('Average training loss per batch is ', 0.18496513319186045)
Avg val

In [62]:
with tf.Session(config=config) as sess:
    print('Trying to load model...')
    try:
        restorer = tf.train.import_meta_graph(save_path + '.meta')
        restorer.restore(sess, tf.train.latest_checkpoint(save_dir))
        print "Model restored"
    except IOError:
        print "No previous model found, running default init"
        sess.run(init)
    print("Init done")
    
    bx = x_train[50:100]
    by = y_train[50:100, 29]
    by = by[:, None]
    op = sess.run(env_model.output, \
            feed_dict={env_model.cur_state:bx,
                       env_model.targets:by,
                       env_model.phase:False})

# print op.shape, by.shape
# for i in zip(np.squeeze(op), np.squeeze(by)):
#     print i

Trying to load model...
Model restored
Init done


In [63]:
np.mean((np.squeeze(op)-np.squeeze(by))**2)

0.11017323824480833

In [64]:
for i in zip(np.squeeze(op), np.squeeze(by)):
    print i

(0.17911743, 0.0)
(0.093046457, 0.0)
(0.13156834, 0.28613931514300006)
(0.075183168, 0.0)
(0.15709624, 1.144557260572)
(0.053191449, 0.28613931513999979)
(0.007503923, -0.28613931513999979)
(0.086839095, 0.0)
(0.059295628, 0.0)
(-0.0029163994, 0.0)
(0.02531508, 0.28613931513999979)
(-0.036283854, 0.0)
(-0.039782193, 0.0)
(-0.10121112, -0.57227863027999981)
(-0.16417766, -0.28613931514799995)
(0.13331677, 0.0)
(0.053894948, 0.0)
(0.21118657, 0.28613931514799995)
(0.16026877, 0.0)
(0.082047328, 0.28613931514000002)
(-0.057933476, 0.0)
(0.035227526, 0.28613931513999979)
(-0.036803689, 0.0)
(-0.12285101, -0.28613931513999979)
(-0.14585981, 0.0)
(-0.14774466, -0.28613931514000002)
(-0.19879135, -0.28613931514799995)
(-0.043731753, 0.0)
(0.027128305, 0.0)
(-0.092596322, -1.144557260572)
(0.24658708, 0.0)
(0.16986302, 0.0)
(0.16923501, 0.0)
(0.14478707, 0.0)
(0.09138447, 0.0)
(0.12362672, 0.0)
(0.057311695, 0.0)
(0.064386889, 0.5722786302859999)
(-0.077138633, 1.144557260574)
(-0.51417947, -0